In [ ]:
!pip install mcpl
!pip install scipy
!pip install mpi4py
!pip install trame
!pip install ipywidgets

In [ ]:
!prompt -g box.gdml -n 100 -v

In [1]:
from cinema_plot import tutorial_run
tutorial_run('cubes.gdml', numNeu=100, showInJupyter=True)

Widget(value="<iframe src='http://localhost:35267/index.html?ui=P_0x7fc246ddc730_0&reconnect=auto' style='widt…